In [3]:
import pandas as pd
import json
import numpy as np

# what i want for a repeatable process:
1. try refetching some lyrics (potentialy with a new strategy)
2. save the ones that worked to disk
3. (repeat)


In [4]:
songs_df = pd.read_csv('./data/1 DONE RIGHT OUTPUT unique songs.csv')
songs_df

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon,['John Lennon'],uncategorized
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,"['Jay-Z', 'Beyonce Knowles']",rock
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis,['Paul Davis'],hip hop
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers,['The Everly Brothers'],country
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson,['Nelson'],uncategorized
...,...,...,...,...,...,...,...
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish,['Billie Eilish'],folk
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty,"['KYLE', 'Lil Yachty']",pop
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole,['J. Cole'],NaN
5064,"my.lifeJ. Cole, 21 Savage & Morray",2,2021-05-29,my.life,"J. Cole, 21 Savage & Morray","['J. Cole', '21 Savage', 'Morray']",NaN


## Ok, new strategy: the original dataset is our source of truth (ie to do joins/merges off of). 

In [5]:
def save_lyrics(lyrics_subset, file_name):
    lyrics_subset.to_csv(f'./data/lyric-fetching-datasets/{file_name}.csv', index=False)
    return lyrics_subset

## Fetching functions

In [6]:
BANNED_CHARACTERS = ['[', ']', '"', "'", '+', ',', '.', '(', ')', '/', '\\', '']
def clean_it(s):
    cleaned = s
    for naughty_char in BANNED_CHARACTERS:
        cleaned = cleaned.replace(naughty_char, '')
    return cleaned.strip()

dirty = 'Whole Wide World (From "True Love"'
clean_it(dirty)

'Whole Wide World From True Love'

In [7]:
import lyricsgenius
GENIUS_CLIEN_API_KEY = '8ZiCNVQqbSF6NM0nwVnVv6Y0EBR7BsVQ9HaVgPdqJOZR1l6XlymR-pUUi6z3GHPx' # be nice :)
genius = lyricsgenius.Genius(GENIUS_CLIEN_API_KEY)

In [8]:
def search_song_chose_first_then_get_lyrics(song, performer):
    try: 
        print(f'{performer} {song}')
        songs = genius.search_songs(clean_it(f'{performer} {song}'))
        urls = [song['result']['url'] for song in songs['hits']]
        return [genius.lyrics(song_url=urls[0]), urls]
    except:
        print(f'OOPS: {performer} {song}')

In [9]:
def fetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: search_song_chose_first_then_get_lyrics(song=row['song'], performer=row['performer']),
        axis=1
    )

In [10]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls
    })
    return save_to_disk

### 1-line demo

In [11]:
convert_fetched_lyrics_to_df(
    fetch_lyrics_and_urls(
        songs_df.head(1)
    ), 
    songs_df.head(1)
)

John Lennon #9 Dream


,song_id,lyrics,urls
0,#9 DreamJohn Lennon,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...,"[""https://genius.com/John-lennon-9-dream-lyric..."


# OK, LET'S JUST GET ALL THE LYRICS WE *HAVE* TRIED FETCHING INTO ONE PLACE (work done previously, in this notebook, & in others)

In [12]:
LYRIC_FOLDER_PATH = './data/lyric-fetching-datasets'

### OK, this is the older fetched stuff, which we already fetched lyrics using (ie a few of these URLS are wrong, & we tried to get better ones, so we should overwrite with those new ones)

## ✅ 1. our big 4,600 row list of succesfully fetched lyrics (some may be wrong, but we'll correct that as we merge others in)

In [13]:
has_lyrics_df = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"has_lyrics"}.csv')
has_lyrics_df

,song_id,lyrics,urls
0,(I Can't Get No) SatisfactionThe Rolling Stones,(I Can’t Get No) Satisfaction Lyrics[Instrumen...,"[""https://genius.com/The-rolling-stones-i-cant..."
1,(I Just) Died In Your ArmsCutting Crew,(I Just) Died in Your Arms Lyrics[Intro]\nOh I...,"[""https://genius.com/Cutting-crew-i-just-died-..."
2,(I Know) I'm Losing YouThe Temptations,(I Know) I’m Losing You Lyrics[Intro]\nOoh\nYo...,"[""https://genius.com/The-temptations-i-know-im..."
3,(I've Had) The Time Of My LifeBill Medley & Je...,(I’ve Had) The Time of My Life Lyrics[Chorus: ...,"[""https://genius.com/Bill-medley-and-jennifer-..."
4,1 ThingAmerie,1 Thing Lyrics[Intro]\nWoo!\nUh\nWoo!\nNa-na-n...,"[""https://genius.com/Amerie-1-thing-lyrics"", ""..."
...,...,...,...
4638,abcdefuGAYLE,TranslationsTürkçeEspañolDeutschFrançais​abcde...,"[""https://genius.com/Gayle-abcdefu-lyrics"", ""h..."
4639,amariJ. Cole,"American Dream Lyrics[Intro: Jeezy]\n(Cryin', ...","[""https://genius.com/Jeezy-american-dream-lyri..."
4640,iSpyKYLE Featuring Lil Yachty,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...","[""https://genius.com/Alex-aiono-i-spy-t-shirt-..."
4641,interludeJ. Cole,"Interlude (Cole World) LyricsI'm drivin', nigg...","[""https://genius.com/J-cole-interlude-cole-wor..."


## ✅ 2. 69 songs where we re-fetched them cuz the lyrics looked obviously wrong

In [14]:
better_url_refetched_lyrics = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"TODO we refetched approx 70 songs with a more likely URL, integrate them"}.csv')
better_url_refetched_lyrics


,song_id,lyrics,urls,likely_correct_url
0,6 Foot 7 FootLil Wayne Featuring Cory Gunz,TranslationsEnglish6 Foot 7 Foot Lyrics[Chorus...,"[""https://genius.com/Genius-users-best-lines-b...",https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
1,679Fetty Wap Featuring Remy Boyz,679 Lyrics[Produced by Peoples]\n\n[Intro: Fet...,"[""https://genius.com/Spotify-new-music-tuesday...",https://genius.com/Fetty-wap-679-lyrics
2,ApologizeTimbaland Featuring OneRepublic,"Apologize Lyrics[Intro: Timbaland]\nAyy, ayy, ...","[""https://genius.com/Nvp-summer-playlist-2015-...",https://genius.com/Timbaland-apologize-lyrics
3,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
4,BillionaireTravie McCoy Featuring Bruno Mars,Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI ...,"[""https://genius.com/Formerly-vinylzombie-mcr-...",https://genius.com/Travie-mccoy-billionaire-ly...
...,...,...,...,...
64,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
65,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
66,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
67,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


In [29]:
(better_url_refetched_lyrics.lyrics.str.len() < 6000).describe()

count       69
unique       2
top       True
freq        58
Name: lyrics, dtype: object

### This is a great sign: there are no longer any obviously too long lyrics (except one, which I added to the instrumental list)

In [30]:
better_url_refetched_lyrics[
    better_url_refetched_lyrics.lyrics.str.len() > 6000
]

,song_id,lyrics,urls,likely_correct_url
3,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
39,Mona LisaLil Wayne Featuring Kendrick Lamar,TranslationsTürkçeРусскийMona Lisa Lyrics[Intr...,"[""https://genius.com/Genius-users-musikalische...",https://genius.com/Lil-wayne-mona-lisa-lyrics
45,Only YouFranck Pourcel's French Fiddles,Grünt #15 Lyrics[Couplet 1 : Alpha Wann]\nPhil...,"[""https://genius.com/Edmund-burke-reflections-...",https://genius.com/Grunt-grunt-15-lyrics


### Ok, but a few songs need to be refetched:

In [38]:
better_url_refetched_lyrics[
    better_url_refetched_lyrics.lyrics.isna()
]

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,NaN,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,NaN,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,NaN,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,NaN,NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,NaN,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
47,PayphoneMaroon 5 Featuring Wiz Khalifa,NaN,NaN,https://genius.com/Maroon-5-payphone-lyrics
50,Rather BeClean Bandit Featuring Jess Glynne,NaN,NaN,https://genius.com/Clean-bandit-rather-be-lyrics
63,UmbrellaRihanna Featuring Jay-Z,NaN,NaN,https://genius.com/Rihanna-umbrella-lyrics


In [41]:
def _get_song(url, urls):
    try: 
        print(url, '...')
        return [genius.lyrics(song_url=url), urls]
    except:
        print(f'OOPS: {url}')
        
def refetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

once_again_with_better_url = refetch_lyrics_and_urls(
    better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]
)
once_again_with_better_url

https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics ...
https://genius.com/Irene-cara-flashdance-what-a-feeling-lyrics ...
https://genius.com/Icona-pop-i-love-it-lyrics ...
https://genius.com/Eve-let-me-blow-ya-mind-lyrics ...
https://genius.com/Maroon-5-moves-like-jagger-lyrics ...
https://genius.com/Maroon-5-payphone-lyrics ...
https://genius.com/Clean-bandit-rather-be-lyrics ...
https://genius.com/Rihanna-umbrella-lyrics ...


11    [TranslationsPortuguêsDrop It Like It’s Hot Ly...
17    [Flashdance... What a Feeling Lyrics[Intro]\nF...
27    [TranslationsPortuguêsI Love It Lyrics[Chorus:...
33    [Let Me Blow Ya Mind Lyrics[Intro: Eve]\nUh, u...
40    [Moves Like Jagger Lyrics[Intro: Adam Levine]\...
47    [TranslationsPortuguêsPayphone Lyrics[Chorus: ...
50    [TranslationsPortuguêsFrançaisRather Be Lyrics...
63    [Umbrella Lyrics[Intro: JAY-Z]\nUh-huh, uh-huh...
dtype: object

In [42]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls, 
        'likely_correct_url': original_df.likely_correct_url
    })
    return save_to_disk

In [43]:
better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,NaN,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,NaN,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,NaN,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,NaN,NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,NaN,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
47,PayphoneMaroon 5 Featuring Wiz Khalifa,NaN,NaN,https://genius.com/Maroon-5-payphone-lyrics
50,Rather BeClean Bandit Featuring Jess Glynne,NaN,NaN,https://genius.com/Clean-bandit-rather-be-lyrics
63,UmbrellaRihanna Featuring Jay-Z,NaN,NaN,https://genius.com/Rihanna-umbrella-lyrics


In [49]:
lol_the_had_to_refetch_a_few_with_new_url = convert_fetched_lyrics_to_df(
    once_again_with_better_url,
    better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]
)

_put_back_in = pd.concat([
    lol_the_had_to_refetch_a_few_with_new_url,
    better_url_refetched_lyrics.dropna()
])

save_lyrics(
    _put_back_in,
    'Refetched with more likely URL (good to go now!)'
)

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,TranslationsPortuguêsDrop It Like It’s Hot Lyr...,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,Flashdance... What a Feeling Lyrics[Intro]\nFi...,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,TranslationsPortuguêsI Love It Lyrics[Chorus: ...,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,"Let Me Blow Ya Mind Lyrics[Intro: Eve]\nUh, uh...",NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,Moves Like Jagger Lyrics[Intro: Adam Levine]\n...,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
...,...,...,...,...
64,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
65,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
66,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
67,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


## ✅ 3. 400 songs that failed to fetch (for many reasons) originally, which I refetched refetched.  🎉 Hurray: of these refetches, all that failed were *instrumentals* -- SIMPLE. I saved a seperate file for those on disk.

In [15]:
big_refetch_of_misc = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"TODO this is the result of refetching 400 non-lyriced songs (some are instrumentals turns out)"}.csv')
big_refetch_of_misc


,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,NaN,NaN
1,(Sittin' On) The Dock Of The BayOtis Redding,(Sittin’ On) The Dock of the Bay Lyrics[Verse ...,"[""https://genius.com/Otis-redding-sittin-on-th..."
2,(You Gotta) Fight For Your Right (To Party!)Be...,Fight for Your Right Lyrics[Intro]\nYeah\nKick...,"[""https://genius.com/Beastie-boys-fight-for-yo..."
3,(You're the) Devil In DisguiseElvis Presley Wi...,NaN,NaN
4,...Baby One More TimeBritney Spears,TranslationsItaliano...Baby One More Time Lyri...,"[""https://genius.com/Britney-spears-baby-one-m..."
...,...,...,...
418,"You're The First, The Last, My EverythingBarry...",NaN,NaN
419,Youngblood5 Seconds Of Summer,TranslationsEspañolPortuguêsYoungblood Lyrics[...,"[""https://genius.com/5-seconds-of-summer-young..."
420,Your LoveThe Outfield,NaN,NaN
421,YummyJustin Bieber,NaN,NaN


In [16]:
IS_INSTRUMENTAL_RETURN_FLAG = 'THIS_IS_AN_INSTRUMENTAL'

In [17]:
_failed_refetch = big_refetch_of_misc[
    big_refetch_of_misc.lyrics.isna()
    & big_refetch_of_misc.urls.notna()
]
_failed_refetch

,song_id,lyrics,urls
8,95.southJ. Cole,NaN,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
10,A Fifth Of BeethovenWalter Murphy & The Big Ap...,NaN,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
27,Axel FHarold Faltermeyer,NaN,"[""https://genius.com/Harold-faltermeyer-axel-f..."
54,Can't Hold UsMacklemore & Ryan Lewis Featuring...,NaN,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
63,Classical GasMason Williams,NaN,"[""https://genius.com/Mason-williams-classical-..."
79,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,NaN,"[""https://genius.com/Rick-dees-and-his-cast-of..."
105,ExodusFerrante & Teicher,NaN,"[""https://genius.com/Ferrante-and-teicher-exod..."
108,Feels So GoodChuck Mangione,NaN,"[""https://genius.com/Chuck-mangione-feels-so-g..."
121,Grazing In The GrassHugh Masekela,NaN,"[""https://genius.com/Hugh-masekela-grazing-in-..."
122,Green OnionsBooker T. & The MG's,NaN,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


In [18]:
_merged_with = songs_df.merge(left_on='song_id', right=_failed_refetch, right_on='song_id')

In [19]:
# this SHOULDN'T TAKE LONG
re_refetched_to_see_instrumentals = fetch_lyrics_and_urls(
    _merged_with
)
re_refetched_to_see_instrumentals

J. Cole 95.south
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/8-bit-arcade-95-south-8-bit-j-cole-emulation-lyrics
Walter Murphy & The Big Apple Band A Fifth Of Beethoven
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Walter-murphy-a-fifth-of-beethoven-lyrics
Harold Faltermeyer Axel F
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Harold-faltermeyer-axel-f-lyrics
Macklemore & Ryan Lewis Featuring Ray Dalton Can't Hold Us
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Cant-hold-us-cant-hold-us-tribute-to-macklemore-ryan-lewis-and-ray-dalton-instrumental-version-lyrics
Mason Williams Classical Gas
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song 

Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Taylor-dayne-tell-it-to-my-heart-dub-of-hearts-mix-lyrics
The Tornadoes Telstar
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/The-tornados-telstar-lyrics
Ramsey Lewis Trio The "In" Crowd
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/The-ramsey-lewis-trio-the-in-crowd-lyrics
Dave "Baby" Cortez The Happy Organ
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Dave-baby-cortez-the-happy-organ-lyrics
Cliff Nobles & Co. The Horse
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Cliff-nobles-and-co-the-horse-lyrics
Herb Alpert & The Tijuana Brass The Lonely Bull (El

0     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
1     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
2     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
3     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
4     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
5     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
6     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
7     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
8     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
9     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
10    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
11    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
12    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
13    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
14    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
15    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
16    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
17    [THIS_IS_AN_INSTRUMENTAL, [https://genius.

In [34]:
# Caught this when looking at lyric length
INSTRUMENTAL_TO_APPEND = {
    'song_id': "Only YouFranck Pourcel's French Fiddles",
    'lyrics': IS_INSTRUMENTAL_RETURN_FLAG,
    'urls': np.nan, 
}

instrumental_formatted_as_df = convert_fetched_lyrics_to_df(
    re_refetched_to_see_instrumentals,
    _merged_with
).append(INSTRUMENTAL_TO_APPEND, ignore_index = True)
instrumental_formatted_as_df

,song_id,lyrics,urls
0,95.southJ. Cole,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
1,A Fifth Of BeethovenWalter Murphy & The Big Ap...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
2,Axel FHarold Faltermeyer,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Harold-faltermeyer-axel-f..."
3,Can't Hold UsMacklemore & Ryan Lewis Featuring...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
4,Classical GasMason Williams,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Mason-williams-classical-..."
5,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Rick-dees-and-his-cast-of..."
6,ExodusFerrante & Teicher,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Ferrante-and-teicher-exod..."
7,Feels So GoodChuck Mangione,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Chuck-mangione-feels-so-g..."
8,Grazing In The GrassHugh Masekela,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Hugh-masekela-grazing-in-..."
9,Green OnionsBooker T. & The MG's,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


In [35]:
save_lyrics(
    instrumental_formatted_as_df,
    'all the INSTRUMENTAL songs'
)

,song_id,lyrics,urls
0,95.southJ. Cole,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
1,A Fifth Of BeethovenWalter Murphy & The Big Ap...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
2,Axel FHarold Faltermeyer,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Harold-faltermeyer-axel-f..."
3,Can't Hold UsMacklemore & Ryan Lewis Featuring...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
4,Classical GasMason Williams,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Mason-williams-classical-..."
5,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Rick-dees-and-his-cast-of..."
6,ExodusFerrante & Teicher,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Ferrante-and-teicher-exod..."
7,Feels So GoodChuck Mangione,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Chuck-mangione-feels-so-g..."
8,Grazing In The GrassHugh Masekela,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Hugh-masekela-grazing-in-..."
9,Green OnionsBooker T. & The MG's,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


_____________________________

# OK, SO EVERYTHING 👇 IS WORK I DID REFETCHING DATA. Everything 👆 is work just processing that data, saving it to disk.

# 1a. Has lyrics! Yay! Done.

In [69]:
# has_lyrics = pd.concat([
#     second_stashed_fetched_lyrics_with_url, stashed_fetched_lyrics_with_url
# ]).dropna(subset=['lyrics']).drop_duplicates(subset=['song_id'])

# save_lyrics(has_lyrics, 'has_lyrics')

In [73]:
has_lyrics_df = pd.read_csv(f'./data/lyric-fetching-datasets/{"has_lyrics"}.csv')
has_lyrics_df

,song_id,lyrics,urls
0,(I Can't Get No) SatisfactionThe Rolling Stones,(I Can’t Get No) Satisfaction Lyrics[Instrumen...,"[""https://genius.com/The-rolling-stones-i-cant..."
1,(I Just) Died In Your ArmsCutting Crew,(I Just) Died in Your Arms Lyrics[Intro]\nOh I...,"[""https://genius.com/Cutting-crew-i-just-died-..."
2,(I Know) I'm Losing YouThe Temptations,(I Know) I’m Losing You Lyrics[Intro]\nOoh\nYo...,"[""https://genius.com/The-temptations-i-know-im..."
3,(I've Had) The Time Of My LifeBill Medley & Je...,(I’ve Had) The Time of My Life Lyrics[Chorus: ...,"[""https://genius.com/Bill-medley-and-jennifer-..."
4,1 ThingAmerie,1 Thing Lyrics[Intro]\nWoo!\nUh\nWoo!\nNa-na-n...,"[""https://genius.com/Amerie-1-thing-lyrics"", ""..."
...,...,...,...
4638,abcdefuGAYLE,TranslationsTürkçeEspañolDeutschFrançais​abcde...,"[""https://genius.com/Gayle-abcdefu-lyrics"", ""h..."
4639,amariJ. Cole,"American Dream Lyrics[Intro: Jeezy]\n(Cryin', ...","[""https://genius.com/Jeezy-american-dream-lyri..."
4640,iSpyKYLE Featuring Lil Yachty,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...","[""https://genius.com/Alex-aiono-i-spy-t-shirt-..."
4641,interludeJ. Cole,"Interlude (Cole World) LyricsI'm drivin', nigg...","[""https://genius.com/J-cole-interlude-cole-wor..."


In [123]:
merged_with_lyrics = pd.merge(
    left=songs_df, 
    left_on='song_id', 
    right=has_lyrics_df, 
    right_on='song_id',
    how='left' # keep all songs
)

merged_with_lyrics['fetched_url'] =  merged_with_lyrics.urls.apply(
    lambda stringified_array: pd.notna(stringified_array) and json.loads(stringified_array)[0]
)
merged_with_lyrics['urls'] =  merged_with_lyrics.urls.apply(
    lambda stringified_array: pd.notna(stringified_array) and json.loads(stringified_array)
)
merged_with_lyrics

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon,['John Lennon'],uncategorized,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...,[https://genius.com/John-lennon-9-dream-lyrics...,https://genius.com/John-lennon-9-dream-lyrics
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,"['Jay-Z', 'Beyonce Knowles']",rock,"’03 Bonnie & Clyde Lyrics[Intro: JAY-Z]\nUhh, ...",[https://genius.com/Jay-z-03-bonnie-and-clyde-...,https://genius.com/Jay-z-03-bonnie-and-clyde-l...
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis,['Paul Davis'],hip hop,’65 Love Affair LyricsI was a car hop\nYou wer...,[https://genius.com/Paul-davis-65-love-affair-...,https://genius.com/Paul-davis-65-love-affair-l...
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers,['The Everly Brothers'],country,(’Til) I Kissed You Lyrics[Verse]\nNever felt ...,[https://genius.com/The-everly-brothers-til-i-...,https://genius.com/The-everly-brothers-til-i-k...
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson,['Nelson'],uncategorized,(Can’t Live Without Your) Love And Affection L...,[https://genius.com/Nelson-cant-live-without-y...,https://genius.com/Nelson-cant-live-without-yo...
...,...,...,...,...,...,...,...,...,...,...
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish,['Billie Eilish'],folk,TranslationsTürkçeBillie Eilish - everything i...,[https://genius.com/Billie-eilish-everything-i...,https://genius.com/Billie-eilish-everything-i-...
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty,"['KYLE', 'Lil Yachty']",pop,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...",[https://genius.com/Alex-aiono-i-spy-t-shirt-i...,https://genius.com/Alex-aiono-i-spy-t-shirt-is...
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole,['J. Cole'],NaN,"Interlude (Cole World) LyricsI'm drivin', nigg...",[https://genius.com/J-cole-interlude-cole-worl...,https://genius.com/J-cole-interlude-cole-world...
5064,"my.lifeJ. Cole, 21 Savage & Morray",2,2021-05-29,my.life,"J. Cole, 21 Savage & Morray","['J. Cole', '21 Savage', 'Morray']",NaN,TranslationsРусскийPortuguêsDeutschItaliano​​m...,[https://genius.com/J-cole-21-savage-and-morra...,https://genius.com/J-cole-21-savage-and-morray...


# 1b. Has lyrics... but they're (probalby) wrong. Fix!

### They don't end in `-lyrics` as most correct URLs do...

In [178]:
url_does_not_end_with_lyrics = merged_with_lyrics[
    merged_with_lyrics.lyrics.notna() &
    (merged_with_lyrics.lyrics != '[]') &
    ~merged_with_lyrics.fetched_url.apply(lambda url: url and url.endswith('-lyrics'))
]

url_does_not_end_with_lyrics

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
42,(You're) Having My BabyPaul Anka with Odia Coates,1,1974-07-06,(You're) Having My Baby,Paul Anka with Odia Coates,['Paul Anka with Odia Coates'],hip hop,Angela’s Ashes LyricsAngela's Ashes\n\nA Memoi...,[https://genius.com/Frank-mccourt-angelas-ashe...,https://genius.com/Frank-mccourt-angelas-ashes...
55,1-800-273-8255Logic Featuring Alessia Cara & K...,3,2017-05-20,1-800-273-8255,Logic Featuring Alessia Cara & Khalid,"['Logic', 'Alessia Cara', 'Khalid']",NaN,"Favorite Cover Songs Lyrics-#-- ""1-800-273-825...",[https://genius.com/Goodapolloiv-favorite-cove...,https://genius.com/Goodapolloiv-favorite-cover...
70,34+35Ariana Grande Feat. Doja Cat & Megan Thee...,2,2021-01-02,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,"['Ariana Grande Feat. Doja Cat', 'Megan Thee S...",uncategorized,Doja Cat [Discography List] LyricsStudio Album...,[https://genius.com/Doja-cat-doja-cat-discogra...,https://genius.com/Doja-cat-doja-cat-discograp...
77,6 Foot 7 FootLil Wayne Featuring Cory Gunz,9,2011-01-01,6 Foot 7 Foot,Lil Wayne Featuring Cory Gunz,"['Lil Wayne', 'Cory Gunz']",uncategorized,Best lines/bars of the 2010's Lyrics Englishha...,[https://genius.com/Genius-users-best-lines-ba...,https://genius.com/Genius-users-best-lines-bar...
78,679Fetty Wap Featuring Remy Boyz,4,2015-07-18,679,Fetty Wap Featuring Remy Boyz,"['Fetty Wap', 'Remy Boyz']",pop,New Music Tuesday 06/30/15 LyricsRefused - Ser...,[https://genius.com/Spotify-new-music-tuesday-...,https://genius.com/Spotify-new-music-tuesday-0...
...,...,...,...,...,...,...,...,...,...,...
4937,You Are So Beautiful/It's A Sin When You Love ...,5,1975-01-11,You Are So Beautiful/It's A Sin When You Love ...,Joe Cocker,['Joe Cocker'],country,Angela’s Ashes LyricsAngela's Ashes\n\nA Memoi...,[https://genius.com/Frank-mccourt-angelas-ashe...,https://genius.com/Frank-mccourt-angelas-ashes...
4977,You Learn/You Oughta KnowAlanis Morissette,6,1996-07-27,You Learn/You Oughta Know,Alanis Morissette,['Alanis Morissette'],pop,Spotify Singles: Complete Collection LyricsThe...,[https://genius.com/Spotify-spotify-singles-co...,https://genius.com/Spotify-spotify-singles-com...
5016,You're Makin' Me High/Let It FlowToni Braxton,1,1996-06-08,You're Makin' Me High/Let It Flow,Toni Braxton,['Toni Braxton'],NaN,Toni Braxton’s 25 Best Songs: Critic’s Picks L...,[https://genius.com/Billboard-toni-braxtons-25...,https://genius.com/Billboard-toni-braxtons-25-...
5040,Young ForeverJay-Z + Mr. Hudson,10,2009-09-26,Young Forever,Jay-Z + Mr. Hudson,['Jay-Z + Mr. Hudson'],country,Die liebsten Closing-Tracks der Community Lyri...,[https://genius.com/Genius-deutschland-die-lie...,https://genius.com/Genius-deutschland-die-lieb...


### ... they literally *all* end in `-annotated` instead, LOL. Found a consistent bug!

In [177]:
url_does_not_end_with_lyrics.fetched_url.apply(lambda s: s[-10:]).describe()

count            245
unique             1
top       -annotated
freq             245
Name: fetched_url, dtype: object

## Ok, so we're going to just refetch the lyrics using the `url` that closely matches the tiltle (if it exists), as measured by `Levenshtein distance` (fancy!)

In [142]:
from Levenshtein import distance

In [163]:
a ='Lil Wayne Featuring Cory Gunz'  
s = '6 Foot 7 Foot'
u = 'https://genius.com/Lil-wayne-6-foot-7-foot-lyrics'
def get_l_dist(performer, song, url):
    format_song = f'{performer.split()[0]} {song} lyrics'
    format_url = url.replace('https://genius.com/', '').replace('-', ' ')
    return distance(
        format_song,
        format_url
    )
# (format_song, format_url)

In [194]:
def return_likely_correct_url_or_else_false(row):
    for url in row['urls']:
        if url.endswith('-lyrics'): 
            dist = get_l_dist(row['performer'], row['song'], url)
            if dist < 13:
                return url
    return np.nan

In [195]:
url_does_not_end_with_lyrics.apply(return_likely_correct_url_or_else_false, axis=1)

42                                                    NaN
55                                                    NaN
70                                                    NaN
77      https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
78                https://genius.com/Fetty-wap-679-lyrics
                              ...                        
4937                                                  NaN
4977                                                  NaN
5016                                                  NaN
5040        https://genius.com/Jay-z-young-forever-lyrics
5047    https://genius.com/Kesha-your-love-is-my-drug-...
Length: 245, dtype: object

In [210]:
with_likely_correct_url = url_does_not_end_with_lyrics.copy()
with_likely_correct_url['likely_correct_url'] = url_does_not_end_with_lyrics.apply(
    return_likely_correct_url_or_else_false, axis=1
)

with_likely_correct_url = with_likely_correct_url.dropna(subset=['likely_correct_url'])

def _get_song(url, urls):
    try: 
        print(url, '...')
        return [genius.lyrics(song_url=url), urls]
    except:
        print(f'OOPS: {url}')
        
def refetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

refeched = with_likely_correct_url.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

https://genius.com/Lil-wayne-6-foot-7-foot-lyrics ...
https://genius.com/Fetty-wap-679-lyrics ...
https://genius.com/Timbaland-apologize-lyrics ...
https://genius.com/Migos-bad-and-boujee-lyrics ...
https://genius.com/Travie-mccoy-billionaire-lyrics ...
https://genius.com/Rae-sremmurd-black-beatles-lyrics ...
https://genius.com/Robin-thicke-blurred-lines-lyrics ...
https://genius.com/Bts-boy-with-luv-lyrics ...
https://genius.com/Katy-perry-california-gurls-lyrics ...
https://genius.com/Fabolous-cant-let-you-go-lyrics ...
https://genius.com/Flo-rida-club-cant-handle-me-lyrics ...
https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics ...
OOPS: https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics
https://genius.com/Jay-z-empire-state-of-mind-lyrics ...
https://genius.com/6ix9ine-fefe-lyrics ...
https://genius.com/Drake-fair-trade-lyrics ...
https://genius.com/Pitbull-feel-this-moment-lyrics ...
https://genius.com/Bruno-mars-finesse-remix-lyrics ...
https://genius.com/Irene-cara-

In [211]:
refeched

77      [TranslationsEnglish6 Foot 7 Foot Lyrics[Choru...
78      [679 Lyrics[Produced by Peoples]\n\n[Intro: Fe...
280     [Apologize Lyrics[Intro: Timbaland]\nAyy, ayy,...
347     [TranslationsPortuguêsРусскийItalianoAzərbayca...
458     [Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI...
                              ...                        
4648    [We Are Young Lyrics[Verse 1: Nate Ruess]\nGiv...
4664    [TranslationsPortuguêsEnglishFrançaisWe Found ...
4670    [TranslationsPortuguêsWe R Who We R Lyrics[Ver...
5040    [Young Forever Lyrics[Intro: Mr. Hudson]\nLet'...
5047    [TranslationsPortuguêsYour Love is My Drug Lyr...
Length: 69, dtype: object

# TODO: update `fetched_url` with `likely_correct_url` used in these calls

In [214]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls, 
        'likely_correct_url': original_df.likely_correct_url
    })
    return save_to_disk

In [216]:
refetched_from_likely_df = convert_fetched_lyrics_to_df(refeched, with_likely_correct_url)

In [218]:
save_lyrics(refetched_from_likely_df, 'TODO we refetched approx 70 songs with a more likely URL, integrate them')

,song_id,lyrics,urls,likely_correct_url
77,6 Foot 7 FootLil Wayne Featuring Cory Gunz,TranslationsEnglish6 Foot 7 Foot Lyrics[Chorus...,"[""https://genius.com/Genius-users-best-lines-b...",https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
78,679Fetty Wap Featuring Remy Boyz,679 Lyrics[Produced by Peoples]\n\n[Intro: Fet...,"[""https://genius.com/Spotify-new-music-tuesday...",https://genius.com/Fetty-wap-679-lyrics
280,ApologizeTimbaland Featuring OneRepublic,"Apologize Lyrics[Intro: Timbaland]\nAyy, ayy, ...","[""https://genius.com/Nvp-summer-playlist-2015-...",https://genius.com/Timbaland-apologize-lyrics
347,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
458,BillionaireTravie McCoy Featuring Bruno Mars,Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI ...,"[""https://genius.com/Formerly-vinylzombie-mcr-...",https://genius.com/Travie-mccoy-billionaire-ly...
...,...,...,...,...
4648,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
4664,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
4670,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
5040,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


# 2. ⚠️ IGNORE FOR NOW ⚠️  Has urls (not lyrics) -- try refetching!

#  3. Needs to be refetched

In [86]:
merged_with_lyrics[
    merged_with_lyrics.lyrics.isna()
]

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
25,(Now and Then There's) A Fool Such As IElvis P...,2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,NaN,NaN,False
29,(Sittin' On) The Dock Of The BayOtis Redding,1,1968-01-27,(Sittin' On) The Dock Of The Bay,Otis Redding,['Otis Redding'],pop,NaN,NaN,False
39,(You Gotta) Fight For Your Right (To Party!)Be...,7,1986-12-20,(You Gotta) Fight For Your Right (To Party!),Beastie Boys,['Beastie Boys'],pop,NaN,NaN,False
41,(You're the) Devil In DisguiseElvis Presley Wi...,3,1963-06-29,(You're the) Devil In Disguise,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,NaN,NaN,False
47,...Baby One More TimeBritney Spears,1,1998-11-21,...Baby One More Time,Britney Spears,['Britney Spears'],hip hop,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...
5026,"You're The First, The Last, My EverythingBarry...",2,1974-11-02,"You're The First, The Last, My Everything",Barry White,['Barry White'],pop,NaN,NaN,False
5046,Youngblood5 Seconds Of Summer,7,2018-06-16,Youngblood,5 Seconds Of Summer,['5 Seconds Of Summer'],rock,NaN,NaN,False
5048,Your LoveThe Outfield,6,1986-02-15,Your Love,The Outfield,['The Outfield'],NaN,NaN,NaN,False
5056,YummyJustin Bieber,2,2020-01-18,Yummy,Justin Bieber,['Justin Bieber'],country,NaN,NaN,False
